In [1]:
import pathlib

data_folderpath = pathlib.Path("./data")

ppd_folderpath = data_folderpath / "uk-ppd"
inflation_filepath = data_folderpath / "uk-ons/ons-inflation-1989-2022.csv"
interest_filepath = data_folderpath / "uk-boe/boe-interest-1975-2022.csv"


In [2]:
from tqdm import tqdm

tqdm.pandas()


In [5]:
import pandas as pd

from typing import Callable


def build_count_properties_sold(ix: pd.DataFrame, n_days: int) -> Callable[[pd.DataFrame], int]:
    def count_properties_sold(row: pd.DataFrame) -> int:
        if row.date and row.postgroup and row.property_type:
            slice = ix.loc[row.date - pd.Timedelta(days=n_days) : row.date, row.postgroup, row.property_type]
            return slice.sum()
        else:
            return None

    return count_properties_sold


# https://www.gov.uk/guidance/about-the-price-paid-data
ppd_property_type = {"D": "detached", "S": "semi-detached", "T": "terraced", "F": "flat/maisonettes"}
ppd_duration = {"F": "freehold", "L": "leasehold"}
ppd_old_or_new = {"Y": "new", "N": "old"}
ppd_cols_src = [
    "id",
    "price",
    "date",
    "postcode",
    "property_type",
    "old_or_new",
    "duration",
    "paon",
    "saon",
    "street",
    "locality",
    "town_city",
    "district",
    "county",
    "ppd_category_type",
    "record_status",
]
ppdf_cols_dst = [
    "date",
    "postgroup",
    "property_type",
    "old_or_new",
    "duration",
    "price",
]
ppd_filepaths = list(ppd_folderpath.glob("*.zip"))
ppd_df = pd.concat([pd.read_csv(fp, names=ppd_cols_src) for fp in tqdm(ppd_filepaths)])

100%|██████████| 5/5 [00:28<00:00,  5.68s/it]


In [24]:
df_missing = pd.DataFrame(ppd_df.isna().sum())
df_missing = df_missing.rename(columns={0: "missing"})
df_missing["%"] = df_missing["missing"].map(lambda x: str(round(x * 100. / len(ppd_df), 2)) + "%")
df_missing = df_missing[df_missing.missing > 0]
df_missing


,missing,%
postcode,18471,0.4%
saon,4013617,87.29%
street,86816,1.89%
locality,2846155,61.9%


In [25]:
df_uniques = pd.DataFrame(ppd_df.drop_duplicates().nunique())
df_uniques = df_uniques.rename(columns={0: "uniques"})
df_uniques["%"] = df_uniques["uniques"].map(lambda x: str(round(x * 100. / len(ppd_df), 2)) + "%")
df_uniques = df_uniques[df_uniques.uniques > 1]
df_uniques

,uniques,%
id,4597894,100.0%
price,96000,2.09%
date,1758,0.04%
postcode,1044446,22.72%
property_type,5,0.0%
old_or_new,2,0.0%
duration,2,0.0%
paon,243571,5.3%
saon,30493,0.66%
street,278907,6.07%


In [23]:
ppd_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4597894 entries, 0 to 1152059
Data columns (total 16 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   id                 object
 1   price              int64 
 2   date               object
 3   postcode           object
 4   property_type      object
 5   old_or_new         object
 6   duration           object
 7   paon               object
 8   saon               object
 9   street             object
 10  locality           object
 11  town_city          object
 12  district           object
 13  county             object
 14  ppd_category_type  object
 15  record_status      object
dtypes: int64(1), object(15)
memory usage: 596.3+ MB
